In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random

import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.model_selection import train_test_split

## Bias-Variance Tradeoff

In modeling we make the assumption that the data follows some unknown function with some noise:

$$
f(X) + \epsilon
$$

Then we create a model $\hat{f}(X)$ that approximates this model as best it can!
- Note that $\epsilon$ is fixed. It represents a barrier to our predictive capabilities in some sense.

The **Bias** of our model $\hat{f}(X)$ is the expected error of our model:
$$
\text{E}[(f(X) - \hat{f}(X)]
$$
However, we don't know $f(X)$ in practice so we often use the loss of our model against the training set as a stand-in for bias. In other words the bias refers to how well the model is at predicting the target variable.

The **Variance** of our model $\text{Var}[\hat{f}(X)]$ represents how sensitive our model is to perturbations in the data.

In [ ]:
# let's use similar fake nonlinear data as when we discussed under/over fitting
num_pts = 100
X = np.linspace(-2, 2, num_pts)

# noise
epsilon = np.random.normal(0, 3, num_pts)

# degree 3 polynomial
Y = 3*(X-1)*(X+2)*(X-1.5) + epsilon

plt.scatter(x=X, y=Y)
plt.show()

- Since the data is non-linear, a linear model has high *Bias* here. We see that the expected error is quite large.
- The linear model also has low *Variance*. If I perturb the input variable $X$, the output is perturbed in a linear fashion.

In [ ]:
model = sm.OLS(Y, sm.add_constant(X), hasconst=True)
res = model.fit()
b, m = res.params
print(b, m)

# plot the points
plt.scatter(x=X, y=Y)

# plot the line
plt.axline((0, b), slope=m, color='green')
plt.show()

Let's fit a high degree polynomial to the data now.

In [ ]:
df = pd.DataFrame({'x':X, 'y':Y})

n = 30

for i in range(n-1):
    df[f'x_{i+2}'] = X**(i+2)
df.head()

In [ ]:
indep_var = 'x'
for i in range(n-1):
    indep_var = indep_var + f' + x_{i+2}'
print(indep_var)

model = ols(formula = f'y ~ {indep_var}', data=df)
res = model.fit()
res.summary()

- Note the "squiggliness" of the model. This model might have decent Bias, but it has high variance as well.
- If we perturb the input variable left or right we might drastically change the model output.

In [ ]:
plt.scatter(x=X, y=Y)

# plot the polynomial
coefs = list(res.params)
coefs.reverse()
poly = [np.polyval(coefs, i) for i in X]
plt.plot(X, poly)

plt.xlim(min(X), max(X))
plt.ylim(min(Y), max(Y))
plt.show()

The **Bias-Variance Tradeoff** is the trade-off between reducing Bias and reducing Variance.
- Often to reduce Bias you have to fit the model more closely to the training data. This might result in a high Variance.
- Often to reduce Variance you need to make the model less sensitive to perturbations. This might result in a higher Bias since you are fitting the training data less closely.

This tradeoff relates to the underfitting vs. overfitting problem. Often high Bias implies underfitting and high Variance implies overfitting.
- The job of the model-er is often to find that "sweet spot" in between!

Techniques that center around reducing the Variance of a model or attempting to reduce the model from overfitting to the training are types of **Regularization**.

## Ridge and Lasso Regression
- These two regularization methods concern the *size* of the coefficients in linear or logistic regression.
- Note the size of the coefficients in the polynomial regression above!

Idea behind reducing the size of the cofficients:
- Makes it harder for the model to overfit, less able to be "squiggly"
- Reduces the effect of multicolinearity, often one variable will "take over" the other.

In [ ]:
# let's take a look at the taxi data
df_taxis = sns.load_dataset('taxis')
df_taxis.head()

In [ ]:
# for fun I'm going to make a new variable
df_taxis['distance_ft'] = df_taxis['distance'] * 5280
df_taxis[['distance', 'distance_ft']].describe()

In [ ]:
# predict distance from total fare
model = ols(formula = 'distance_ft ~ total + passengers', data=df_taxis)
res = model.fit()
res.summary()

- Why are the values so large?
- Why might it be a bad idea to limit the size of the coefficients here?

In [ ]:
df_taxis[['distance_ft', 'passengers', 'total']].boxplot(figsize=(15,7)) 
plt.show()

Often **Normalizing** the data prevents issues such as this. It also may drastically improve your model as variables at different scales can make numerical optimization techniques less effective.

To normalize a variable $X$ we create a new variable

$$
X' = \frac{X - \mu}{\sigma}
$$

where $\mu, \sigma$ are the mean and standard deviation of $X$ respectively.

In [ ]:
df = df_taxis.copy()[['distance', 'distance_ft', 'passengers', 'total']]
df_taxis_normalized = (df - df.mean()) / df.std()
df_taxis_normalized.describe()

In [ ]:
df_taxis_normalized.boxplot(figsize=(15,7)) 
plt.show()

- Notice what happens to distance vs. distance_ft when normalized!
- Note that the $R^2$ below did not change, but the coefficients did.

In [ ]:
model = ols(formula = 'distance_ft ~ total + passengers', data=df_taxis_normalized)
res = model.fit()
res.summary()

You can also do min-max normalization to put everything on a $[0,1]$ scale.

In [ ]:
df = df_taxis.copy()[['distance', 'distance_ft', 'passengers', 'total']]
df_taxis_normalized = (df - df.min()) / (df.max() - df.min())
df_taxis_normalized.boxplot(figsize=(15,7)) 
plt.show()

### WARNING
- If we do a train-test split why is it data leakage to normalize the variables against the *entire* dataset??
- Always remember to use the same normalization method/parameters at prediction time!

### Ridge Regression

- Idea: Add a penalty term to the loss function. For example when there are $N$ datapoints and $n$ independent variables.

$$
\begin{align}
\mathcal{L} &= \left(\sum_{i=1}^N (y_i - \hat{y}_i)^2\right) + \lambda \left(\sum_{i=0}^n \beta_0^2\right)\\
&= RSS + \lambda ||\beta||_2^2
\end{align}
$$
- Here $\lambda$ controls the magnitude of the penalty

In [ ]:
df_pen = sns.load_dataset('penguins').dropna().reset_index(drop=True)
df_pen.head()

In [ ]:
train, val = train_test_split(df_pen, test_size=0.95, random_state=2022)

In [ ]:
# normalize the numerical columns
for col in ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']:
    
    train_mean = train[col].mean()
    train_std = train[col].std()
    
    train[col] = ((train[col] - train_mean) / train_std)
    val[col] = ((val[col] - train_mean) / train_std)

train.head()

In [ ]:
# An Aside: see what happens if I give the model a categorical variable
model = ols(formula = 'bill_depth_mm ~ bill_length_mm + flipper_length_mm + body_mass_g', data=train)
res = model.fit()
res.summary()

In [ ]:
y_pred = res.predict(val)

# score the model
y = val['bill_depth_mm']
y_mean = train['bill_depth_mm'].mean()  # use the mean of the training set

TSS = sum((y - y_mean)**2)
RSS = sum((y - y_pred)**2)
print(f'This model has an R^2 on the validation set of {(TSS - RSS) / TSS}')

In [ ]:
# very small training set
len(train)

In [ ]:
# remember there is some multicolinearity as well
df_pen.corr()

In [ ]:
# sklearn has a nice ridge regression model
from sklearn.linear_model import Ridge

x_train = train[['bill_length_mm', 'flipper_length_mm', 'body_mass_g']]
y_train = train['bill_depth_mm']

model = Ridge(alpha=10.0)
model.fit(x_train,y_train)

y_pred = model.predict(x_train)

# score the model
r2 = model.score(x_train, y_train)
print(f'This model has an R^2 on the train set of {r2}')

In [ ]:
x_val = val[['bill_length_mm', 'flipper_length_mm', 'body_mass_g']]
y_val = val['bill_depth_mm']

r2 = model.score(x_val, y_val)
print(f'This model has an R^2 on the validation set of {r2}')

In [ ]:
# also known as hyperparamter tuning
for alpha in [0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100]:
    
    # create and evaluate the model for different values of alpha
    model = Ridge(alpha=alpha)
    model.fit(x_train,y_train)
    r2 = model.score(x_val, y_val)
    
    print(f'This model has an R^2 on the validation set of {r2} when alpha is {alpha}')

### Lasso Regression

- Idea: Add a different penalty term to the loss function. For example when there are $N$ datapoints and $n$ independent variables.

$$
\begin{align}
\mathcal{L} &= \left(\sum_{i=1}^N (y_i - \hat{y}_i)^2\right) + \lambda \left(\sum_{i=0}^n |\beta_0|\right)\\
&= RSS + \lambda ||\beta||_1
\end{align}
$$
- Here $\lambda$ controls the magnitude of the penalty

In [ ]:
# sklearn has a nice ridge regression model
from sklearn.linear_model import Lasso

x_train = train[['bill_length_mm', 'flipper_length_mm', 'body_mass_g']]
y_train = train['bill_depth_mm']

model = Lasso(alpha=0.01)
model.fit(x_train,y_train)

y_pred = model.predict(x_train)

# score the model
r2 = model.score(x_train, y_train)
print(f'This model has an R^2 on the train set of {r2}')

In [ ]:
x_val = val[['bill_length_mm', 'flipper_length_mm', 'body_mass_g']]
y_val = val['bill_depth_mm']

r2 = model.score(x_val, y_val)
print(f'This model has an R^2 on the validation set of {r2}')

In [ ]:
for alpha in [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1]:
    
    # create and evaluate the model for different values of alpha
    model = Lasso(alpha=alpha)
    model.fit(x_train,y_train)
    r2 = model.score(x_val, y_val)
    
    print(f'This model has an R^2 on the validation set of {r2} when alpha is {alpha}')

### Ridge vs. Lasso

- Both apply a penalty based on the magnitude of the coefficients
- Ridge tends to lower the magnitude of all the coefficients somewhat equally
- Lasso tends to "zero out" some of the coefficients (often is used for feature selection)
- Check out this article [here](https://explained.ai/regularization/index.html) by Terence Parr, former Professor at USF who is now at Google.

In [ ]:
# loop over many different alpha and see the coefficients
rows = []
for alpha in [0.001, 0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]:
    
    # create and evaluate the model for different values of alpha
    model = Ridge(alpha=alpha)
    model.fit(x_train,y_train)
    
    row = {'alpha': alpha,
           'intercept': model.intercept_}
    for i, coef in enumerate(model.coef_):
        row[f'beta_{i}'] = coef
    rows.append(row)
    
pd.DataFrame(rows)

In [ ]:
# loop over many different alpha and see the coefficients
rows = []
for alpha in [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1]:
    
    # create and evaluate the model for different values of alpha
    model = Lasso(alpha=alpha)
    model.fit(x_train,y_train)
    
    row = {'alpha': alpha,
           'intercept': model.intercept_}
    for i, coef in enumerate(model.coef_):
        row[f'beta_{i}'] = coef
    rows.append(row)
    
pd.DataFrame(rows)

### Cross-Validation

- How do we find the best alpha? What if our choice of validation set biased which alpha was the best?
- How do we get a less biased estimation of a metric?
- Idea: Split the data into disjoint pieces, then train on the complement of the piece and score on the piece.
- $k$-fold CV: Split data into $k$ equal pieces. Create and score $k$ models.

In [ ]:
# let's do 9-fold cross-validation with basic Linear Regression
from sklearn.linear_model import LinearRegression

# shuffle the data
df = df_pen.sample(frac=1, random_state=2020)

rows = []
for i in range(9):
    
    # 9 equal splits 
    val = df[i*(37):(i+1)*37]
    train = df[~df.index.isin(val.index)]
    
    x_train = train[['bill_length_mm', 'flipper_length_mm', 'body_mass_g']]
    y_train = train['bill_depth_mm']
    
    x_val = val[['bill_length_mm', 'flipper_length_mm', 'body_mass_g']]
    y_val = val['bill_depth_mm']

    model = LinearRegression()
    model.fit(x_train, y_train)
    
    r2 = model.score(x_val, y_val)
    print(r2)
    row = {'R^2' : model.score(x_val, y_val)}
    rows.append(row)
    
df = pd.DataFrame(rows)
print('The Average R^2 is', df['R^2'].mean())

In [ ]:
from sklearn.model_selection import GridSearchCV

model = Ridge()

# can pass other parameters as well here!
params = {'alpha' : [0.001, 0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]}

# define the search
search = GridSearchCV(model, params, scoring='r2', cv=9)

# shuffle the data, GridSearch does not shuffle
df = df_pen.sample(frac=1, random_state=2020)
x = df[['bill_length_mm', 'flipper_length_mm', 'body_mass_g']]
y = df['bill_depth_mm']

# execute search
result = search.fit(x, y)

In [ ]:
result.best_score_, result.best_params_

- How many models did the above grid search create?